# Column Generation


In [4]:
import time

from exputils.extent.cg import calculate_extent_CG
from exputils.state.random_ket import make_random_quantum_state

In [5]:
seed = 0
kind = "pure"

for n in [4, 5, 6, 7, 8, 9]:
    print(f"{n=}")
    psi = make_random_quantum_state(kind, n, seed)
    t0 = time.perf_counter()
    stabilizer_extent, extends, max_values, _ = calculate_extent_CG(
        n, psi, verbose=False
    )
    t1 = time.perf_counter()
    print(f"{stabilizer_extent=} {t1-t0=}")

    with open(f"../data/CG/{kind}_{n}_data.txt", mode="w") as f:
        for i in range(len(extends)):
            print(f"{extends[i]} {max_values[i]}", file=f)

    with open(f"../data/CG/{kind}_{n}_time.txt", mode="w") as f:
        print(t1 - t0, file=f)

n=4
CG: n=4, method='mosek'
start: calculate dots
iteration: 1 / 100, Amat.shape = (16, 10000)
start: solve SOCP
stabilizer_extent=2.543604607010636
start: calculate dual dots
OPTIMAL!
stabilizer_extent=2.543604607010636 t1-t0=2.1311435624957085
n=5
CG: n=5, method='mosek'
start: calculate dots
iteration: 1 / 100, Amat.shape = (32, 10000)
start: solve SOCP


KeyboardInterrupt: 